In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from model_resnet import ResNet
from model_resnet import ResidualBlock

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 80
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True)

model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

Files already downloaded and verified


In [2]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(), acc))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.6812 | Accuracy: 32.0100
Epoch [1/80], Step [200/500] Loss: 1.4535 | Accuracy: 37.6850
Epoch [1/80], Step [300/500] Loss: 1.1692 | Accuracy: 41.8433
Epoch [1/80], Step [400/500] Loss: 1.2613 | Accuracy: 44.8875
Epoch [1/80], Step [500/500] Loss: 1.2141 | Accuracy: 47.5000
Epoch [2/80], Step [100/500] Loss: 0.9878 | Accuracy: 60.0700
Epoch [2/80], Step [200/500] Loss: 1.1662 | Accuracy: 60.8850
Epoch [2/80], Step [300/500] Loss: 0.9443 | Accuracy: 61.8967
Epoch [2/80], Step [400/500] Loss: 0.8571 | Accuracy: 62.7475
Epoch [2/80], Step [500/500] Loss: 1.0103 | Accuracy: 63.5640
Epoch [3/80], Step [100/500] Loss: 0.9639 | Accuracy: 67.7000
Epoch [3/80], Step [200/500] Loss: 0.9534 | Accuracy: 68.0900
Epoch [3/80], Step [300/500] Loss: 0.7992 | Accuracy: 68.5700
Epoch [3/80], Step [400/500] Loss: 0.8599 | Accuracy: 68.9375
Epoch [3/80], Step [500/500] Loss: 0.6415 | Accuracy: 69.3560
Epoch [4/80], Step [100/500] Loss: 0.8664 | Accuracy: 72.1300
Epoch [4

In [3]:
# 1. Loading CIFAR dataset
test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)

model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the pre-trained model on the test images: {} %'.format(100 * correct / total))

Files already downloaded and verified
Accuracy of the pre-trained model on the test images: 88.28 %


In [5]:
"""
Now Train more than 80epochs
"""

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(), acc))
            
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Accuracy of the pre-trained model on the test images: {} %'.format(100 * correct / total))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 0.1009 | Accuracy: 95.9700
Epoch [1/80], Step [200/500] Loss: 0.1657 | Accuracy: 95.8300
Epoch [1/80], Step [300/500] Loss: 0.0944 | Accuracy: 95.8067
Epoch [1/80], Step [400/500] Loss: 0.1819 | Accuracy: 95.8675
Epoch [1/80], Step [500/500] Loss: 0.0753 | Accuracy: 95.8540
Accuracy of the pre-trained model on the test images: 88.47 %
Epoch [2/80], Step [100/500] Loss: 0.1803 | Accuracy: 95.9300
Epoch [2/80], Step [200/500] Loss: 0.1960 | Accuracy: 95.9850
Epoch [2/80], Step [300/500] Loss: 0.0896 | Accuracy: 96.0500
Epoch [2/80], Step [400/500] Loss: 0.0817 | Accuracy: 95.9350
Epoch [2/80], Step [500/500] Loss: 0.2178 | Accuracy: 95.9220
Accuracy of the pre-trained model on the test images: 88.17 %
Epoch [3/80], Step [100/500] Loss: 0.1464 | Accuracy: 96.0200
Epoch [3/80], Step [200/500] Loss: 0.1187 | Accuracy: 95.8850
Epoch [3/80], Step [300/500] Loss: 0.1114 | Accuracy: 95.8500
Epoch [3/80], Step [400/500] Loss: 0.1413 | Accuracy: 95.9075
Epoch [3

KeyboardInterrupt: 

In [6]:
# 장소 옮기느라 꺼져서 epoch 111부터 다시 ㄱㄱ

"""
Now Train more than 80epochs
"""

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(), acc))
            
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Accuracy of the pre-trained model on the test images: {} %'.format(100 * correct / total))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 0.1515 | Accuracy: 93.4300
Epoch [1/80], Step [200/500] Loss: 0.1555 | Accuracy: 93.5000
Epoch [1/80], Step [300/500] Loss: 0.1298 | Accuracy: 93.4767
Epoch [1/80], Step [400/500] Loss: 0.2177 | Accuracy: 93.3625
Epoch [1/80], Step [500/500] Loss: 0.2313 | Accuracy: 93.3780
Accuracy of the pre-trained model on the test images: 85.88 %
Epoch [2/80], Step [100/500] Loss: 0.1321 | Accuracy: 93.7100
Epoch [2/80], Step [200/500] Loss: 0.2329 | Accuracy: 93.8100
Epoch [2/80], Step [300/500] Loss: 0.3096 | Accuracy: 93.5700
Epoch [2/80], Step [400/500] Loss: 0.4027 | Accuracy: 93.5775
Epoch [2/80], Step [500/500] Loss: 0.2310 | Accuracy: 93.5380
Accuracy of the pre-trained model on the test images: 87.66 %
Epoch [3/80], Step [100/500] Loss: 0.2188 | Accuracy: 93.5900
Epoch [3/80], Step [200/500] Loss: 0.3494 | Accuracy: 93.5450
Epoch [3/80], Step [300/500] Loss: 0.3041 | Accuracy: 93.5167
Epoch [3/80], Step [400/500] Loss: 0.1115 | Accuracy: 93.6000
Epoch [3